# The 0-1 Knapsack Problem solved with a VQE

[The Knapsack Problem](https://en.wikipedia.org/wiki/Knapsack_problem) is an optimzation problem that goes like this: You are given a list of items that each has a weight and a value. You also have a knapsack that can hold a maximum weigh. Determine which items to take in the knapsack so as to maximize the total value taken without going over the maiximum weight.

The most efficient approach would be a greedy approach, but that is not guaranteed to give the best result. For example let's take the these items
 - a tv of weight 12 and value 75
 - a watch of weight 3 and value 50
 - a vase of weight 10 and value 40
 - a ball of weight 5 and value 10
 
Taking the most valuable product (the tv and the watch) would give us a total value of 125. Getting the lightest objects (the watch, the ball and the vase) gives us a value 100. Getting the items with the best value to weight ratio (the watch and the tv) gives us 125. The best choice is to take the tv, the watch and the ball with a total value of 135.

## The VQE approach

The VQE algorithm can find the ground state of an Ising Hamiltonian. In order to solve an optimization problem using this algorithm we need to 
 1. first express the solution to our problem as a binary string. 
 2. Then we need to find a cost function that acts on binary strings and that reachs it minimal value at the point wehere the binary string is the solution to the optimization problem. I.e. If we minimize the cost function and get the input to the minimum value that input is our solution. 
 3. We then need to map our function to an Ising Hamiltonian
 4. Use the VQE Algorithm to find the ground state of that Hamiltonian

We are done - the ground state that we found is the solution to our problem.

Fortunately for step 3 there is a cookie-cutter technique to mapping a cost function to an Ising Hamiltonian and step 4 is built into Qiskit. So we really only need to worry about step 1 and 2.

### The cost function

For the 0-1 Knapsack Problem the solution can be expressed as an array $X =[x_{0}..x_{n-1}]$ of 0s and 1s in the following way: Each item is represented at an index of the array. If the value is 1 then we take the item in the knapsac. If it's 0 we don't take it.

We can represent our weights as an array $W =[w_{0}..w_{n-1}]$ and the values as an array $V =[v_{0}..v_{n-1}]$ and name the maximum weight $ W_{max} $. 

To build the cost function we must keep in mind that we want to maximize the sum $ \sum_{i} x_{i}v_{i} $ while comlying with the constraint that $ W_{max} >= \sum_{i} x_{i}w_{i} $. This cost function is difficult to express so we use a mathematical trick. We add another term  $ S $ that will vary with our solution. In order to make it change wit the solution we enhance $ X $ with a number of aditional bits $ X' = [x_0,..x_{n-1},y_{n}..y_{n+m-1}]  $. The bytes $ y_i $ will be the binary representation of $ S $. In this way the optimizer will be able to optimize $ S $ as well as $ X $

We can now write out cost function

$$ C(X') = M(W_{max} - \sum_{i=0}^{n-1} x_{i}w_{i} - S)^2 - \sum_{i}^{n-1} x_{i}v_{i} $$ where $ S = \sum_{j=n}^{n+m} 2^jy_{j} $ and $M$ is a number large enought to dominate the sum of values. 

We see that, because $S$ can only be positive, when $W_{max} >= \sum_{i} x_{i}w_{i}$ the optimizer can find an $S$ (or better the $y_j$ that compose $S$) so that it will take the first term to 0. This way the function is dominated by the sum of values. If $W_{max} < \sum_{i} x_{i}w_{i}$ then the first term can never be 0 and, multiplied by a large $M$, will always dominate the function.
The minimum value of the function will be that where the constraint is respected and the sum of values is maximized.

Note that because $[y_n..y_{n_m-1}]$ is the binary representation of $S$ and $S$ is at most $W_{max}$ them $m = \log(W_{max})$


### Mapping to an Ising Hamiltonian

To map this function to a Hamiltonian we use a known technique. We first expand the first term. We then ignore the constant terms since they cannot be minimized. We then take each term of the remaining sum and apply the mapping $x_i\rightarrow (I-Z_i)/2$, where $I$ is the identity matrix of order $n+m$ and $Z_i$ is the matrix that results from tensoring $n+m$ matrixes of which at every positin other than #i# the factors of the tensor are identity matrixes ans at position $i$ is the Pauly $Z$ matrix.

This is the cookie-cutter technique I mentioned earlier. And luckily we don't even need to do the tensor products ourselves. Qiskit has an object that takes the inde of the Pauly $Z$ in the tensor and does the rest.

Afer doing this replacement we expand again and remove the constant terms. The rest we need to return as a list of terms to the VQE algorithm.

### Getting the solution

The VQE minimizes the function and gives us $X'$. All we need to do is thake the first $n$ terms and we have our solution



In [1]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua import run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from collections import OrderedDict
import math

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging

Defining the terms

In [82]:
def get_knapsack_operator(values, weights, max_weight):
    if len(values) != len(weights):
        raise ValueError("The values and weights must have the same length")

    if any(v < 0 for v in values) or any(w < 0 for w in weights):
        raise ValueError("The values and weights cannot be negative")

    if all(v == 0 for v in values):
        raise ValueError("The values cannot all be 0")

    if max_weight < 0:
        raise ValueError("max_weight cannot be negative")

    y_size = int(math.log(max_weight, 2)) + 1 if max_weight > 0 else 1
    print(y_size)
    n = len(values)
    num_values = n + y_size
    pauli_list = []
    shift = 0

    M = 2000000  #10 * np.sum(values)

    # term for sum(x_i*w_i)**2
    for i in range(n):
        for j in range(n):
            coefficient = -1 * 0.25 * weights[i] * weights[j] * M

            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[j] = not zp[j]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[i] = not zp[i]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

            coefficient = -1 * coefficient
            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[i] = not zp[i]
            zp[j] = not zp[j]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

    # term for sum(2**j*y_j)**2
    for i in range(y_size):
        for j in range(y_size):
            coefficient = -1 * 0.25 * (2 ** i) * (2 ** j) * M

            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[n + j] = not zp[n + j]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[n + i] = not zp[n + i]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

            coefficient = -1 * coefficient
            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[n + i] = not zp[n + i]
            zp[n + j] = not zp[n + j]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

    # term for -2*W_max*sum(x_i*w_i)
    for i in range(n):
        coefficient = max_weight * weights[i] * M

        xp = np.zeros(num_values, dtype=np.bool)
        zp = np.zeros(num_values, dtype=np.bool)
        zp[i] = not zp[i]
        pauli_list.append([coefficient, Pauli(zp, xp)])
        shift -= coefficient

    # term for -2*W_max*sum(2**j*y_j)
    for j in range(y_size):
        coefficient = max_weight * (2 ** j) * M

        xp = np.zeros(num_values, dtype=np.bool)
        zp = np.zeros(num_values, dtype=np.bool)
        zp[n + j] = not zp[n + j]
        pauli_list.append([coefficient, Pauli(zp, xp)])
        shift -= coefficient

    for i in range(n):
        for j in range(y_size):
            coefficient = -1 * 0.5 * weights[i] * (2 ** j) * M

            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[n + j] = not zp[n + j]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[i] = not zp[i]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

            coefficient = -1 * coefficient
            xp = np.zeros(num_values, dtype=np.bool)
            zp = np.zeros(num_values, dtype=np.bool)
            zp[i] = not zp[i]
            zp[n + j] = not zp[n + j]
            pauli_list.append([coefficient, Pauli(zp, xp)])
            shift -= coefficient

    # term for sum(x_i*v_i)
    for i in range(n):
        coefficient = 0.5 * values[i]

        xp = np.zeros(num_values, dtype=np.bool)
        zp = np.zeros(num_values, dtype=np.bool)
        zp[i] = not zp[i]
        pauli_list.append([coefficient, Pauli(zp, xp)])
        shift -= coefficient

    return WeightedPauliOperator(paulis=pauli_list), shift


def get_solution(x, values):
    return x[:len(values)]


def knapsack_value_weight(solution, values, weights):
    value = np.sum(solution * values)
    weight = np.sum(solution * weights)
    return value, weight

def sample_most_likely(state_vector):
    if isinstance(state_vector, dict) or isinstance(state_vector, OrderedDict):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x


Checking the solution with an exact eigensolver

In [130]:
# values = [680, 120, 590, 178]
# weights = [13, 6, 15, 9]
# w_max = 32

values = [8, 2]
weights = [3, 1]
w_max = 3

qubitOp, offset = get_knapsack_operator(values, weights, w_max)

algo_input = EnergyInput(qubitOp)
ee = ExactEigensolver(qubitOp, k=1)
result = ee.run()

most_lightly = result['eigvecs'][0]
x = sample_most_likely(most_lightly)

print(x)
print('result=' + str(x[:len(values)]))

2
[1. 0. 0. 0.]
result=[1. 0.]


Running with a statevector simulator

In [134]:
seed = 10598

spsa = SPSA(max_trials=10000)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

result_statevector = vqe.run(quantum_instance)

most_lightly_sv = result_statevector['eigvecs'][0]
x_statevector = sample_most_likely(most_lightly_sv)

print(x_statevector)
print('result usig statevector_simulator =' + str(x_statevector[:len(values)]))

[0. 1. 0. 1.]
result usig statevector_simulator =[0. 1.]


In [68]:
# run quantum algorithm with shots
seed = 10598

spsa = SPSA(max_trials=1000)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result_shots = vqe.run(quantum_instance)

most_lightly_shots = result_shots['eigvecs'][0]
x_shots = sample_most_likely(most_lightly_shots)

print(x_shots)
print('result usig qasm_simulator =' + str(x_shots[:len(values)]))

[0 0 0 0 0 0 1 0 0 1]
result usig qasm_simulator =[0 0 0 0]


In [43]:
math.log(8, 2)

3.0